In [1]:
import plotly.express as px

In [13]:
# Load the labels
def load_labels(filename: str) -> dict:
    
    dict_storage = {}
    
    with open(filename) as file:
        for index, line in enumerate(file):
            dict_storage[index] = line.strip().split(' ')
    
    return dict_storage

dialog_act_dict = load_labels('dailydialog/dialogues_act.txt')
emotion_dict = load_labels('dailydialog/dialogues_emotion.txt')

In [14]:
# Validation

for index, act_labels in dialog_act_dict.items():
    
    emotion_labels = emotion_dict[index]
    
    if len(emotion_labels) != len(act_labels):
        print(index)

In [26]:
# Mappers

act_mapper = {
    '1': 'inform', '2': 'question', 
    '3': 'directive', '4': 'commissive'
}

emotion_mapper = {
    '0': 'neutral', '1': 'anger', '2': 'disgust', 
    '3': 'fear', '4': 'happiness', '5': 'sadness', 
    '6': 'surprise'
}

def translate_labels(act_labels: list) -> list:
    return [act_mapper[act] for act in act_labels]

def translate_emotion_labels(emotion_labels: list) -> list:
    return [emotion_mapper[emotion] for emotion in emotion_labels]

# DailyDialog Dataset Analysis

This analysis is divided into the following parts:

- Dialog Act and Emotion labels
- Utterance-related
- Topic-related

# Dialog Act and Emotion labels

The following sections looks at the visualizations, in relation to dialog act and emotion labels

The visualizations are:


In [25]:
# Knowledge interaction graph for dialog act
knowledge_graph_dict = {
    'START': {},
    'inform': {},
    'question': {},
    'directive': {},
    'commissive': {}
}

def perform_mapping(act_labels: list, knowledge_graph: dict) -> dict:
    
    for index, label in enumerate(act_labels):
        
        # First label is always 'START'
        if index == 0:
            knowledge_graph['START'][label] = \
                knowledge_graph['START'].get(label, 0) + 1
        
        # Last label is always 'END' and break
        if index == len(act_labels) - 1:
            knowledge_graph[label]['END'] = \
                knowledge_graph[label].get('END', 0) + 1
            break
        
        # Fetch the next label
        # Find the connection
        next_label = act_labels[index + 1]
        knowledge_graph[label][next_label] = \
            knowledge_graph[label].get(next_label, 0) + 1
    
    return knowledge_graph

for index, act_labels in dialog_act_dict.items():

    # Translate labels
    act_labels = translate_labels(act_labels)
    knowledge_graph_dict = perform_mapping(act_labels, knowledge_graph_dict)

print(knowledge_graph_dict)

{'START': {'directive': 3113, 'question': 6469, 'inform': 3531, 'commissive': 5}, 'inform': {'inform': 17843, 'END': 8308, 'directive': 5598, 'question': 14238, 'commissive': 545}, 'question': {'inform': 21817, 'question': 3277, 'directive': 3894, 'END': 405, 'commissive': 35}, 'directive': {'commissive': 9015, 'inform': 983, 'question': 3765, 'END': 1424, 'directive': 2108}, 'commissive': {'END': 2981, 'directive': 2582, 'inform': 2358, 'question': 1679, 'commissive': 125}}


In [27]:
# If current utterance is neutral and the next utterance is non-neutral
# Find the dialog act of the current utterance and the next utterance

# The 'Key' is the neutral emotion, while the inner 'Keys' are the non-emotions
emotion_trigger_dict = {
    'inform': {},
    'question': {},
    'directive': {},
    'commissive': {}
}

def find_the_triggers(act_labels, emotion_labels, trigger_dict):
    
    for index, emotion in emotion_labels.items():
        
        # Check if this is the end
        if index == len(emotion_labels) - 1:
            break
            
        # Check if next emotion is non-neutral
        next_emotion = emotion[index + 1]
        
        # Condition: current emotion is neutral, next emotion is not
        if emotion == 'neutral' and next_emotion != 'neutral':
            
            current_dialog_act = act_labels[index]
            next_dialog_act = act_labels[index + 1]
            
            print(f'Current emotion is {emotion} with act {})
            
            trigger_dict[current_dialog_act][next_dialog_act] = \
                knowledge_graph[current_dialog_act].get(next_dialog_act, 0) + 1
            
    return trigger_dict

for index, act_labels in dialog_act_dict.items():
    
    # Translate the labels
    act_labels = translate_labels(act_labels)
    
    # Translate the emotions
    emotion_labels = translate_emotion_labels(emotion_dict[index])
    
    print(act_labels)
    print(emotion_labels)
    break

['directive', 'commissive']
['disgust', 'neutral']


In [ ]:
# If current utterance is non-neutral and the next utterance is neutral
# Find the dialog act of the current utterance and the next utterance